# EE 379K Lab 3
## Rohan Nagar and Wenyang Fu

# Question 1: Jaccard Similarity

In [6]:
def jaccard_similarity(x, y, z):
    intersection_cardinality = len(set.intersection(*[set(x), set(y), set(z)]))
    union_cardinality = len(set.union(*[set(x), set(y), set(z)]))
    return intersection_cardinality / float(union_cardinality)

jaccard_similarity(['nike', 'running', 'shoe'],
                   ['nike', 'black', 'running', 'shoe'],
                   ['nike', 'blue', 'jacket', 'adidas'])

0.14285714285714285

# Question 2: Minhash

### Part A: Create the characteristic matrix with the alphabet as the seven words {'nike', 'running', 'shoe', 'black', 'blue', 'jacket', 'adidas'}

### Part B: For a random permutation of the seven alphabet elements, find a way to compute the first non-zero element of each column (each set) under the permutation.

### Part C: Instead of choosing a random permutation, use the hash function $h(x) = 3x + 2 (mod 7)$.

### Part D: Generate your own hash functions of the form $h(x) = ax + b (mod 7)$ by choosing $a$ and $b$ at random from the set $\{0, 1, ..., 6\}$. Do this twenty times to estimate the Jaccard Similarity of the three sets. How closely do you approximate the true values, computed in question one?

# Question 3: Implementing Minhash

### Repeat the above where instead of permuting the entire characteristic matrix, implement the algorithm described in Chapter 3 of MMDS for implementing Minhash.

# Question 4: More Minhash (Shingling)

### Part A: Load the 5 article excerpts in $\tt{Lab3articles-5.txt}$.

### Part B: Use $\tt{stopwords}$ from $\tt{nltk.corpus}$ to strip the stopwords from the 5 articles.

### Part C: Compute the $k$-shingles of the documents for $k = 2$, where you shingle on the words, not letters.

### Part D: Compute the $k$-shingles of the documents for $k = 3$, where you shingle on the characters, not words.

# Question 5: Even More Minhash (Document Similarity)

### Part A

### Part B